In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/flywire

In [5]:
import numpy as np
import scipy as sp
from utility import M,F, benchmark_mapping, get_best_connected, invert, score, get_highest_out_degrees
rng = np.random.default_rng(0)

In [6]:
m,n = 50,100
H_M,m_labels = get_best_connected(M,n)
H_F,f_labels = get_best_connected(F,n)
H_M = H_M.todense()
H_F = H_F.todense()
H_F[np.arange(n),np.arange(n)] = 0
H_M[np.arange(n),np.arange(n)] = 0
np.count_nonzero(H_M)/n**2, np.count_nonzero(H_F)/n**2

(0.4681, 0.243)

In [40]:
def find_best_move(G,H, mapping, u, mask = None):

    m = G.shape[0]
    u_row = G[u,:]
    u_rnz = np.flatnonzero(u_row)
    u_row = u_row[u_rnz]
    u_col = G[:,u]
    u_cnz = np.flatnonzero(u_col)
    u_col = u_col[u_cnz]

    if mask is None:
        n = H.shape[0]
        X = H[:,mapping[None,u_rnz]].reshape((n,len(u_rnz)))
        Y = H[mapping[u_cnz,None],:].reshape((len(u_cnz),n))
    else:
        X = H[mask,mapping[None,u_rnz]]
        n = X.shape[0]
        X = X.reshape((n,len(u_rnz)))
        Y = H[mapping[u_cnz,None],mask].reshape((len(u_cnz),n))

    scores = np.minimum(u_row[None,:],X).sum(axis=1) + np.minimum(u_col[:,None],Y).sum(axis=0)
    scores -= np.minimum(G[u,u],H.diagonal())

    return scores

#The mask says which x in [n] are valid choices for mapping[u]. G and H must have 0-diagonal
def make_best_swap(G,H, mapping, u, scores_by_vertex, mask = None):

    if mask is None:
        m = G.shape[0]
        n = H.shape[0]
    else:
        in_image = np.zeros(len(mask),dtype=bool)
        in_image[mapping] = True
        H_mask = np.logical_and(in_image,mask)
        G_mask = mask[mapping]
        m = np.count_nonzero(G_mask)
        n = np.count_nonzero(H_mask)

    row = G[u,:]
    rnz = np.flatnonzero(row)
    row = row[rnz]
    col = G[:,u]
    cnz = np.flatnonzero(col)
    col = col[cnz]

    if mask is None:
        X = H[mapping[:,None],mapping[None,rnz]].reshape((n,len(rnz)))
        Y = H[mapping[cnz,None],mapping[None,:]].reshape((len(cnz),n))
    else:
        X = H[H_mask,mapping[None,rnz]].reshape((n,len(rnz)))
        Y = H[mapping[cnz,None],H_mask].reshape((len(cnz),n))

    u_scores = np.minimum(row[None,:],X).sum(axis=1) + np.minimum(col[:,None],Y).sum(axis=0)

    f_u = mapping[u]

    row = H[f_u,mapping]
    rnz = np.flatnonzero(row)
    row = row[rnz]
    col = H[mapping,f_u]
    cnz = np.flatnonzero(col)
    col = col[cnz]

    if mask is None:
        X = G[:,rnz[None,:]].reshape((m,len(rnz)))
        Y = G[cnz[:,None],:].reshape((len(cnz),m))
    else:
        X = G[G_mask,rnz[None,:]].reshape((m,len(rnz)))
        Y = G[mapping[cnz,None],G_mask].reshape((len(cnz),m))

    f_u_scores = np.minimum(row[None,:],X).sum(axis=1) + np.minimum(col[:,None],Y).sum(axis=0)

    if mask is None:
        scores = u_scores + f_u_scores + np.minimum(G[u,:],H[mapping,f_u])+np.minimum(G[:,u],H[f_u,mapping])
        scores -= scores_by_vertex
        scores -= scores_by_vertex[u]
        #The actual change will be 2*scores[w], but this does not change which vertex we will choose for the swap.
    else:
        scores = u_scores + f_u_scores + np.minimum(G[u,G_mask],H[mapping[G_mask],f_u])+np.minimum(G[G_mask,u],H[f_u,mapping[G_mask]])
        scores -= scores_by_vertex[G_mask]
        scores -= scores_by_vertex[u]

    w = scores.argmax()
    if scores[w] > 0:

        f_w = mapping[w]

        scores_by_vertex += np.minimum(G[:,u],H[mapping,f_w])+np.minimum(G[u,:],H[f_w,mapping])+np.minimum(G[:,w],H[mapping,f_u])+np.minimum(G[w,:],H[f_u,mapping])
        scores_by_vertex -= np.minimum(G[:,u],H[mapping,f_u])+np.minimum(G[u,:],H[f_u,mapping])+np.minimum(G[:,w],H[mapping,f_w])+np.minimum(G[w,:],H[f_w,mapping])

        scores_by_vertex[u] = u_scores[w]+ np.minimum(G[u,w],H[f_w,f_u])+np.minimum(G[w,u],H[f_u,f_w])
        scores_by_vertex[w] = f_u_scores[w]+ np.minimum(G[u,w],H[f_w,f_u])+np.minimum(G[w,u],H[f_u,f_w])

        mapping[u] = f_w
        mapping[w] = f_u

        return scores[w]//2
    
    else:
        return 0
    
def get_scores_by_vertex(G,H,mapping):

    m = G.shape[0]
    scores = np.minimum(G,H[mapping[:,None],mapping[None,:]])
    return scores.sum(axis=0)+scores.sum(axis=1)

In [10]:
result_matrix = np.zeros((n,n),dtype=int)
score_track = []
mapping_save = []

for k in range(500):

    mapping_M = rng.choice(n,m,replace=False)
    mapping_F = rng.choice(n,m,replace=False)

    for i in range(50):

        outer_unchanged = True

        if i % 2 == 0:
            G = H_F[mapping_F[:,None],mapping_F[None,:]]
            H = H_M
            mapping = mapping_M.copy()
        else:
            G = H_M[mapping_M[:,None],mapping_M[None,:]]
            H = H_F
            mapping = mapping_F.copy()

        unmapped = np.ones(n,dtype=bool)
        unmapped[mapping] = False
        top_score = score(G,H,mapping,require_surjective=False)
        
        for j in range(50):

            unchanged = True

            for u in rng.permutation(m):

                change_scores = find_best_move(G,H,mapping,u)
                w = (change_scores*unmapped).argmax()

                if change_scores[w] > change_scores[mapping[u]]:
                    top_score = top_score + change_scores[w]-change_scores[mapping[u]]

                    unmapped[mapping[u]] = True
                    unmapped[w] = False
                    mapping[u] = w
                    unchanged = False
                    outer_unchanged = False

            if unchanged:
                break
        
        if i % 2 == 0:
            mapping_M = mapping.copy()
        else:
            mapping_F = mapping.copy()

        if outer_unchanged:
            print(f"{k=}, {i=}, {top_score=}"+" "*20,end="\r")
            break

    A = H_F[mapping_F[:,None],mapping_F[None,:]]
    B = H_M[mapping_M[:,None],mapping_M[None,:]]

    score_track.append((score(G,H,mapping,require_surjective=False), score(A,A,np.arange(m)), score(B,B,np.arange(m))))
    mapping_save.append((mapping_F,mapping_M))
    result_matrix[mapping_F,mapping_M] += 1

KeyboardInterrupt: 

In [ ]:
np.concatenate([m_labels[mapping_M,None],f_labels[mapping_F,None]],axis=1)

In [36]:
A=M.todense()
B=F.todense()
N=A.shape[0]
A[np.arange(N),np.arange(N)]=0
B[np.arange(N),np.arange(N)]=0

In [37]:
G= B
H=A
mapping = rng.permutation(N)
score(G,H,mapping)

45197

In [41]:
scores_by_vertex = get_scores_by_vertex(G,H,mapping)
score_track=score(G,H,mapping)

for u in np.arange(N):
    improvement = make_best_swap(G,H,mapping,u,scores_by_vertex)
    score_track += improvement
    print(f"{u=}, {score_track=}",end="\r")

scores_by_vertex.sum()/2, score(G,H,mapping)

KeyboardInterrupt: 

In [57]:
def greedy_max(A, B, start_mapping = None, similarity = None, c=0.1, verbose = True, injective = True):
    n = A.shape[0]

    if start_mapping is None:
        mapping = -np.ones(n,dtype=int)
        unmapped_source = np.ones(n,dtype=bool)
        unmapped_target = np.ones(n,dtype=bool)
    else:
        mapping = start_mapping.copy()
        unmapped_source = mapping < 0
        unmapped_target = np.ones(n,dtype=bool)
        unmapped_target[mapping[np.logical_not(unmapped_source)]] = False

    priority = c*np.log1p(A.sum(axis=1))

    if similarity is None:
        if start_mapping is None:
            similarity = np.minimum(A.sum(axis=1).reshape(-1,1),B.sum(axis=1).reshape(1,-1))
            similarity += np.minimum(A.sum(axis=0).reshape(-1,1),B.sum(axis=0).reshape(1,-1))
            similarity = np.log1p(similarity)
        else:
            similarity = np.zeros((n,n),dtype=np.int64)
            for u in np.flatnonzero(np.logical_not(unmapped_source)):
                v = mapping[u]

                u_out = np.flatnonzero(A[u,:])
                u_out = u_out[unmapped_source[u_out]]
                v_out = np.flatnonzero(B[v,:])  
                v_out = v_out[unmapped_target[v_out]]

                min_out = np.minimum.outer(A[[u],u_out],B[[v],v_out])
                np.add.at(similarity,(u_out[:,None],v_out[None,:]),min_out)

                u_in = np.flatnonzero(A[:,u])
                u_in = u_in[unmapped_source[u_in]]
                v_in = np.flatnonzero(B[:,v])
                v_in = v_in[unmapped_target[v_in]]

                min_in = np.minimum.outer(A[u_in,[u]],B[v_in,[v]])
                np.add.at(similarity,(u_in[:,None],v_in[None,:]),min_in)

            priority = similarity.max(axis=1)
            priority[np.logical_not(unmapped_source)] = -1
            if injective:
                similarity[:,np.logical_not(unmapped_target)] = -1


    for i in range(np.count_nonzero(unmapped_source)):

        u = priority.argmax()
        v = similarity[u,:].argmax()
        
        u_out = np.flatnonzero(A[u,:])
        v_out = np.flatnonzero(B[v,:])  
        if injective:
            u_out = u_out[unmapped_source[u_out]]
            v_out = v_out[unmapped_target[v_out]]

        min_out = np.minimum.outer(A[[u],u_out],B[[v],v_out])
        np.add.at(similarity,(u_out[:,None],v_out[None,:]),min_out)

        u_in = np.flatnonzero(A[:,u])
        v_in = np.flatnonzero(B[:,v])

        if injective:
            u_in = u_in[unmapped_source[u_in]]
            v_in = v_in[unmapped_target[v_in]]

        min_in = np.minimum.outer(A[u_in,[u]],B[v_in,[v]])
        np.add.at(similarity,(u_in[:,None],v_in[None,:]),min_in)

        unmapped_source[u] = False
        unmapped_target[v] = False
        mapping[u]=v

        u_out = u_out[unmapped_source[u_out]]
        v_out = v_out[unmapped_target[v_out]]
        u_in = u_in[unmapped_source[u_in]]
        v_in = v_in[unmapped_target[v_in]]
        if (len(u_out)>0 and len(v_out)>0):
            priority[u_out] = np.maximum(priority[u_out],similarity[u_out[:,None],v_out[None,:]].argmax(axis=1))
        if (len(u_in)>0 and len(v_in)>0):
            priority[u_in] = np.maximum(priority[u_in],similarity[u_in[:,None],v_in[None,:]].argmax(axis=1))
        if injective:
            similarity[:,v] = -1
            similarity[u,:] = -1
            
        priority[u] = -1

        if verbose:
            print(f"{i=}",end="\r")

    return mapping

In [5]:
rng = np.random.default_rng()
from algorithms import greedy_mapping
#start_mapping=-np.ones(N,dtype=int)
inverse_benchmark = invert(benchmark_mapping)
#start_mapping[f_labels[mapping_F]]=m_labels[mapping_M]
#random_set = rng.choice(N,500,replace=False)
#start_mapping[:500] = inverse_benchmark[:500]
mapping = greedy_max(H_F,H_M,start_mapping=None, c=0.1)

In [34]:
results = np.zeros((n,n),dtype=int)
for i in range(n):
    for j in range(n):
        if i==j:
            continue
        start_mapping = -np.ones(n,dtype=int)
        start_mapping[0]=i
        start_mapping[1]=j
        mapping=greedy_max(H_F,H_M,start_mapping,verbose=False,c=0)
        results[i,j] = score(H_F,H_M,mapping)
        print(f"{i=}",end="\r")

In [87]:
m_labels=inverse_benchmark[f_labels]
H_M = (M[m_labels[:,None],m_labels[None,:]]).todense()
score(H_F,H_M,np.arange(n))

11233

In [91]:
from algorithms import random_swaps

mapping = rng.permutation(n)
G=H_F
H=H_M
high_score = score(G,H,mapping,require_surjective=False)
last_change = 0
save_count = 8

for i in range(100):
    start_mapping = -np.ones(n,dtype=int)
    #choice = np.concatenate([np.arange(100),rng.choice(np.arange(100,n),size=save_count,replace=False)])
    choice = rng.choice(n,size=save_count,replace=False)
    start_mapping[choice]=mapping[choice]

    next_mapping=greedy_max(G,H,start_mapping,verbose=False,c=0.1,injective=False)
    random_swaps(G,H,next_mapping,5000,verbose=False)

    current_score = score(G,H,next_mapping,require_surjective=False)

    if current_score>high_score:
        high_score=current_score
        mapping = next_mapping
        last_change = i
    elif i-last_change >10:
        save_count += 1
        last_change = i

    print(f"{i=}, {high_score=}, {save_count=}",end="\r")

mapping

array([60, 51,  7,  7, 60, 51, 45,  7, 70, 91, 64, 51, 51, 60, 70, 70, 70,
       51, 64, 70, 64, 91, 70, 64, 45, 60, 70, 60, 52, 51, 44, 60, 70, 70,
       64, 91, 60,  7, 64, 70,  7, 60, 91, 70, 51, 51, 91, 51, 64, 60, 51,
        7, 91, 60, 60,  7, 70, 91, 64, 64, 64,  7,  7,  7, 64, 70, 86, 51,
       91, 60, 51, 60, 60, 51, 51, 51, 51, 64, 64,  7, 44, 70, 70, 64,  7,
       64, 64, 44,  7, 45, 91, 91, 51,  7, 64, 60, 51, 91, 70,  7])

In [ ]:
m=50
mapping_M = rng.choice(n,m,replace=False)
mapping_F = rng.choice(n,m,replace=False)

for i in range(50):

    outer_unchanged = True

    if i % 2 == 0:
        G = H_F[mapping_F[:,None],mapping_F[None,:]]
        H = H_M
        mapping = mapping_M.copy()
    else:
        G = H_M[mapping_M[:,None],mapping_M[None,:]]
        H = H_F
        mapping = mapping_F.copy()

    unmapped = np.ones(n,dtype=bool)
    unmapped[mapping] = False
    top_score = score(G,H,mapping,require_surjective=False)
    
    for j in range(50):

        unchanged = True

        for u in rng.permutation(m):

            change_scores = find_best_move(G,H,mapping,u)
            w = (change_scores*unmapped).argmax()

            if change_scores[w] > change_scores[mapping[u]]:
                top_score = top_score + change_scores[w]-change_scores[mapping[u]]

                unmapped[mapping[u]] = True
                unmapped[w] = False
                mapping[u] = w
                unchanged = False
                outer_unchanged = False

        if unchanged:
            break
    
    if i % 2 == 0:
        mapping_M = mapping.copy()
    else:
        mapping_F = mapping.copy()

    if outer_unchanged:
        print(f"{i=}, {top_score=}"+" "*20,end="\r")
        break